In [4]:
import numpy as np
import pandas as pd 
import plotly.graph_objects as go
import json
import os 
import sys 

In [7]:
filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/gw170817/TransientTable_gw170817.csv"
#filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/gw170817/TransientTable_gw170817.json"

dataframe = pd.read_csv(filename, sep=";")
dataframe.head()

,Unnamed: 0,ampel_z,ampel_z_precision,ampel_dist,possis_abspeak,possis_chisq,possis_ndof,chisquares,grb_spatial,kilonovaness,...,grb_temporal,map_area,map_dist,map_dist_unc,map_name,trigger_time,map_seed,stock,ztf_id,channels
0,5,0.00968,0.003,9.947793,-15.228597,4.568042,4,"{'bns_maurice_1': {'chisq': 6.097292984824451,...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",14,...,"({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",27.814716,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,gw170817
1,4,0.00968,0.003,9.947793,-15.280613,3.231773,3,{'bns_maurice_1': {'chisq': 1.2498391714150732...,"({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",14,...,"({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",27.814716,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,gw170817
2,3,0.00968,0.003,9.947793,-15.312410,1.918000,2,{'bns_maurice_1': {'chisq': 0.4834567925625450...,"({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",14,...,"({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",27.814716,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,gw170817
3,6,0.00968,0.003,9.947793,-15.290809,11.847952,5,"{'bns_maurice_1': {'chisq': 253.4549498628877,...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",14,...,"({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",27.814716,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,gw170817
4,9,0.00968,0.003,9.947793,-15.003717,31.216407,8,"{'bns_maurice_1': {'chisq': 708.592238624895, ...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",14,...,"({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",27.814716,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,gw170817


In [8]:
def jsonify(string: str):
    tmp_str = string.replace("'", '"')
    tmp_str = tmp_str.replace("None", "null")
    tmp_str = tmp_str.replace("True", "true")
    tmp_str = tmp_str.replace("False", "false")

    return tmp_str

In [25]:
risedec_df = pd.DataFrame()

pos_rowslist_dict = {"bns_maurice_1": [], "bns_maurice_2": [], "bns_maurice_3": [], "bns_maurice_4": []}

rowslist = []
for k, row in dataframe.iterrows():
    dec_str = jsonify(row["rise_decline"])
    #print(tmp_str)
    dec_dict = json.loads(dec_str)
    #print(dec_dict)

    rowslist.append(dec_dict)

    pos_str = jsonify(row["chisquares"])
    pos_dict = json.loads(pos_str)
    #print(pos_dict)
    for key, value in pos_dict.items():
        value["ndet"] = dec_dict["ndet"]
        value["latest_det"] = dec_dict["jd_last"]
        value["first_det"] = dec_dict["jd_det"]
        pos_rowslist_dict[key].append(value)
    #print(row["rise_decline"]["ndet"])

risedec_df = pd.DataFrame(rowslist)

pos_df_dict = {}
for key in pos_rowslist_dict:
    pos_df_dict[key] = pd.DataFrame(pos_rowslist_dict[key]).sort_values(by="ndet")


In [27]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(go.Scatter(x=pos_df_dict[key]["latest_det"] - pos_df_dict[key]["first_det"], y = pos_df_dict[key]["chisq"]/pos_df_dict[key]["ndof"], name=key))

fig.update_layout({"xaxis_title": "days from first detection",
                   "yaxis_title": "Reduced chisqu"})
fig.show()

In [29]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(go.Scatter(x=pos_df_dict[key]["latest_det"] - pos_df_dict[key]["first_det"], y = pos_df_dict[key]["obspeak_model_B"], name=key))

fig.update_layout({"xaxis_title": "days from first detection",
                   "yaxis_title": "Reduced chisqu"})
fig.show()